In [ ]:
import logging
from itertools import product
from datetime import datetime, timedelta

import shapely
import hvplot.pandas
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr

from searvey import usgs

logging.basicConfig(
    level=20,
    style="{",
    format="{asctime:s}; {levelname:8s}; {threadName:23s}; {name:<25s} {lineno:5d}; {message:s}",
)

logging.getLogger("urllib3").setLevel(30)
logging.getLogger("parso").setLevel(30)

logger = logging.getLogger(__name__)

## Retrieve Station Metadata

In [ ]:
usgs_stations = usgs.get_usgs_stations()
usgs_stations

In [ ]:
plot_df = usgs_stations.drop_duplicates(subset='site_no').dropna(subset=['site_no', 'dec_lat_va', 'dec_long_va'])
world_plot = plot_df.hvplot(geo=True, tiles=True, hover_cols=["site_no", "location"])
world_plot.opts(width=800, height=500)

In [ ]:
usgs_stations.columns

## Retrieve station metadata from arbitrary polygon

In [ ]:
us_northeast = shapely.geometry.box(-75, 35, -60, 40)
us_northeast

ne_stations = usgs.get_usgs_stations(region=us_northeast)
ne_stations

In [ ]:
ne_stations[ne_stations.begin_date > "2022"]

## Retrieve USGS station data

In [ ]:
ne_data = usgs.get_usgs_data(
    usgs_metadata=ne_stations,
    endtime="2020-01-30",
    period=3,
)
ne_data

In [ ]:
def drop_all_nan_coords(ds: xr.Dataset) -> xr.Dataset:
    for coord in ds.coords:
        ds = ds.isel({
            coord: np.where(
                ds.value.notnull().sum([dim for dim in ds.coords if dim != coord])
            )[0]
        })

    return ds

ds = ne_data.isel(code=ne_data.name.str.contains('Tidal')).sel(option='').squeeze().reset_coords()
ds = drop_all_nan_coords(ds)
ds

In [ ]:
ds.to_dataframe().value.dropna().hvplot(by='site_no', grid=True)